In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib

from scipy import signal
from scipy.signal import find_peaks
from scipy.integrate import cumtrapz

In [ ]:
# サンプリングレート(0.015秒に1回)
sampling_rate = 66

filter_num = sampling_rate * 1

In [ ]:
class Data:
    def __init__(self, folder_name):
        self.folder_name = 'data/' + folder_name
        self.path = folder_name
        self.file_names = os.listdir(self.folder_name)

        # acceleration.csvがない場合は作成する
        if 'acceleration.csv' not in self.file_names:
            self.make_acceleration()
            self.file_names = os.listdir(self.folder_name)
        
        # 'all.csv'がある場合は削除する
        if 'all.csv' in self.file_names:
            self.file_names.remove('all.csv')
        
        self.df = pd.DataFrame()
        for file_name in self.file_names:
            df_tmp = pd.read_csv(os.path.join(self.folder_name, file_name))
            file_type = file_name.replace('.csv', '')

            # 列名を変更してファイル名を追加
            df_tmp = df_tmp.rename(columns={'x': file_type + '_x', 'y': file_type + '_y', 'z': file_type + '_z'})
            
            # 'rotate.csv' に 'w' 列がある場合は削除
            if 'w' in df_tmp.columns and file_name == 'rotate.csv':
                df_tmp = df_tmp.drop(['w'], axis=1)

            if type(df_tmp['time'][0]) == int:
                # 'time'列の変換を実行
                df_tmp['time'] = df_tmp['time'] / 1000
                df_tmp['time'] = pd.to_datetime(df_tmp['time'], unit='s')

            # df_tmpをdfに追記する,既にある列名は追記しない
            self.df = pd.concat([self.df, df_tmp], axis=1, sort=False)

            # 呼び出したファイルに上書き保存する
            df_tmp.to_csv(os.path.join(self.folder_name, file_name), index=False)

        # 1行目のtime列以外のtime列を削除する
        self.df = self.df.iloc[:, [0] + [i for i in range(1, len(self.df.columns)) if not self.df.columns[i].startswith('time')]]
        self.df['time'] =pd.to_datetime(self.df['time']/1000,unit='s')

        # csvファイルを出力する
        self.df.to_csv(os.path.join(self.folder_name, 'all.csv'), index=False)

    def make_acceleration(self):
        df_tmp = pd.read_csv(os.path.join(self.folder_name, 'velocity.csv'))
        df_acc = df_tmp
        # df_tmpからx,y,z列を削除する
        df_acc = df_acc.drop(['x', 'y', 'z'], axis=1)
        df_acc['x'] = df_tmp['x'].cumsum()
        df_acc['y'] = df_tmp['y'].cumsum()
        df_acc['z'] = df_tmp['z'].cumsum()
        df_acc.to_csv(os.path.join(self.folder_name, 'acceleration.csv'), index=False)
        df_acc = df_acc.drop(['time'],axis =1 )
        df_acc['acceleration_x'] = df_acc['x']
        df_acc['acceleration_y'] = df_acc['y']
        df_acc['acceleration_z'] = df_acc['z']
        df_acc = df_acc.drop(['x', 'y', 'z'], axis=1)
        self.df = pd.concat([self.df, df_acc], axis=1, sort=False)
        return self

    def low_filter(self, column_type, n):
        if column_type == 'all':
            for column in self.file_names:
                column = column.replace('.csv', '')
                if column == 'all':
                    pass
                self.low_filter(column, n)
        else:
            column = column_type + '_x'
            out_column = column_type + '_low_x'
            self.df[out_column] = self.df[column].rolling(n).mean()
            column = column_type + '_y'
            out_column = column_type + '_low_y'
            self.df[out_column] = self.df[column].rolling(n).mean()
            column = column_type + '_z'
            out_column = column_type + '_low_z'
            self.df[out_column] = self.df[column].rolling(n).mean()
        return self
    
    def diff(self, column_type):
        if column_type == 'all':
            for column in self.file_names:
                column = column.replace('.csv', '')
                if column == 'all':
                    pass
                self.diff(column)
        else:
            column = column_type + '_x'
            out_column = column_type + '_diff_x'
            self.df[out_column] = self.df[column].diff()
            column = column_type + '_y'
            out_column = column_type + '_diff_y'
            self.df[out_column] = self.df[column].diff()
            column = column_type + '_z'
            out_column = column_type + '_diff_z'
            self.df[out_column] = self.df[column].diff()
        return self

    def plot(self, column_type ,option, axis):
        self.i = 0
        self.fig = plt.figure(figsize=(15, 25))
        
        if column_type == 'all':
            # 'all.csv'がある場合は削除する
            for column in self.file_names:
                column = column.replace('.csv', '')
                if column == 'all':
                    continue
                self.ax(column, option,axis)
                self.i += 1
        else:
            column_type = column_type.replace('.csv', '')
            self.ax(column_type, option,axis)
        plt.show()

    def ax(self, column_type ,option,axis):
        self.low_filter(column_type, filter_num)
        ax = self.fig.add_subplot(5, 1, self.i+1)
        if option == "diff":
            column_type = column_type + '_diff'
        if option == "low":
            column_type = column_type + '_low'  
        match axis:
            case 'x':
                ax.plot(self.df['time'], self.df[column_type + '_x'], label = column_type + '_x')
            case 'y':
                ax.plot(self.df['time'], self.df[column_type + '_y'], label = column_type + '_y')
            case 'z':
                ax.plot(self.df['time'], self.df[column_type + '_z'], label = column_type + '_z')
            case _:
                ax.plot(self.df['time'], self.df[column_type + '_x'], label = column_type + '_x')
                ax.plot(self.df['time'], self.df[column_type + '_y'], label = column_type + '_y')
                ax.plot(self.df['time'], self.df[column_type + '_z'], label = column_type + '_z')
        ax.legend()
        # タイトル
        ax.set_title(column_type)
        ax.set_xlabel('time')
        ax.set_ylabel(column_type)
        ax.grid()

    def split_time(self, start_time, end_time):
        self.df = self.df[(self.df['time'] >= start_time) & (self.df['time'] <= end_time)]
        return self

In [ ]:
# oculus_stairsのデータを読み込む
oculus_stairs = Data('oculus_stairs')
oculus_stairs.low_filter('all', filter_num)

In [ ]:
# 時間を制限する
start_time = '2023-10-24 11:01:20'
end_time = '2023-10-24 11:02:15'
# end_time = '2023-10-24 11:01:38'
oculus_stairs.split_time(start_time, end_time)

In [ ]:
# dfのpositionを微分する
oculus_stairs.diff('position')
oculus_stairs.low_filter('position_diff', filter_num)
oculus_stairs.plot('position', 'diff','y')

In [ ]:
# oculus_stairsのposition_diff_yからpeekを取得して、そのpeekの時間を取得する
peek, _ = signal.find_peaks(oculus_stairs.df['position_diff_y'], distance=20, height=0.004)

df = pd.DataFrame()
df['time'] = oculus_stairs.df['time']

# indexの初期値を取得する
peek_first = oculus_stairs.df.index[0]

# ハイパスフィルターをかける
df['position_y'] = oculus_stairs.df['position_y'] - oculus_stairs.df['position_y'].rolling(filter_num*2,center = False).mean()

# 0.5以上の時は1、0.5以下の時は0にする
df['position_pro_y'] = df['position_y'].apply(lambda x: 1 if x > 0.05 else 0)

# timeをずらす
df['shift_time'] = df['time'] - pd.Timedelta(seconds=0.3)

# 1の時のみのdfを作成する
stair_up_df = pd.DataFrame()
stair_up_df = df[df['position_pro_y'] == 1]

# print(peek)
# indexの初期値を取得する
peek_first = oculus_stairs.df.index[0]
# peekに初期値を足す
peek = peek + peek_first
# peekの時間を取得する
peek_time = oculus_stairs.df['time'][peek]
# plotする
fig = plt.figure(figsize=(15, 5))
plt.plot(df['time'], df['position_y'])
plt.plot(df['shift_time'], df['position_pro_y'])
# plt.scatter(peek_time, oculus_stairs.df['position_diff_y'][peek], s=30, color='red', zorder=2)
plt.title('position_diff_y')
plt.xlabel('time')
plt.grid()
plt.show()

In [ ]:
# position_pro_yとpositionのplot
fig = plt.figure(figsize=(15, 5))
ax = fig.add_subplot(1, 1, 1)
ax.plot(oculus_stairs.df['time'], oculus_stairs.df['position_x'])
ax.plot(oculus_stairs.df['time'], oculus_stairs.df['position_y'])
ax.plot(oculus_stairs.df['time'], oculus_stairs.df['position_z'])
# 後ろを全体で塗る
ax.fill_between(df['shift_time'], -2, 7, where=(df['position_pro_y'] == 1), facecolor='blue', alpha=0.2)
ax.set_title('階段を登っている部分のラベル付け')
ax.set_xlabel('time')
ax.grid()
plt.show()

In [ ]:
oculus_stairs.plot('all', '','')